# DADO LISTADO DE LICITACIONES ETIQEUTADAS GO-NoGO GENERAL MODELO

http://www.mercadopublico.cl/fichaLicitacion.html?idLicitacion=5546-3-LP23 ---> Ficha de licitaciones

In [1]:
import pandas as pd
import numpy as np

from pandas import json_normalize
from datetime import datetime, timedelta
import requests
import json
from os import wait
import time



In [2]:
!pip install tensorflow
import tensorflow as tf


import pandas as pd
import spacy
!python -m spacy download es_core_news_sm # Download the Spanish language model

In [3]:
  global MydirData

  MydirData= "/content/drive/MyDrive/UDD_Data_Science/CasoUso/data/"

In [4]:
# Parto cargando my Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)  # si es que ya lo habia montado, fuerzo el punto de montaje



Mounted at /content/drive


# Carga data desde un archivo

In [59]:
# leer archivo csv

id_retoma = "13500"
#myfile = MydirData + 'licitaciones' + id_retoma + '.csv'
#df = pd.read_csv(myfile, encoding="utf-8")

myfile = MydirData + 'licitaciones' + id_retoma + '_1.xlsx'
df = pd.read_excel(myfile)

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47492 entries, 0 to 47491
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CodigoExterno    47486 non-null  object 
 1   Nombre           47492 non-null  object 
 2   CodigoEstado     47492 non-null  int64  
 3   FechaCierre      47482 non-null  object 
 4   Descripcion      13426 non-null  object 
 5   CodigoCategoria  13426 non-null  float64
 6   Categoria        13426 non-null  object 
 7   CodigoProducto   13426 non-null  float64
 8   NombreProducto   13426 non-null  object 
 9   CodigoOrganismo  13426 non-null  float64
 10  NombreOrganismo  13426 non-null  object 
 11  RutUnidad        13426 non-null  object 
 12  ComunaUnidad     12381 non-null  object 
 13  RegionUnidad     13426 non-null  object 
 14  go               545 non-null    object 
dtypes: float64(3), int64(1), object(11)
memory usage: 5.4+ MB


In [61]:
df.shape

(47492, 15)

# Modelo red neuronal profunda con TensorFlow/Keras

In [62]:
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.optimizers import Adam
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler

import nltk
from nltk.corpus import stopwords


In [63]:
# Se escala Codigo Producto y Categoria
scaler = StandardScaler()
df["CodigoProducto"] = scaler.fit_transform(df[["CodigoProducto"]])
df["CodigoCategoria"] = scaler.fit_transform(df[["CodigoCategoria"]])
df["Texto_full"] = df["Nombre"].astype(str) + " " + df["Descripcion"].astype(str)
df["go"] = df["go"].replace({"x": 1, "NO": 0})

#reemplazar bull por 0 en CodigoCategoria
df["CodigoCategoria"] = df["CodigoCategoria"].replace(np.nan, 0)
df["CodigoProducto"] = df["CodigoProducto"].replace(np.nan, 0)

# eliminar los vacios cuando Go es null
df = df.dropna(subset=["go"])


<ipython-input-63-332e96fe9d49>:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["go"] = df["go"].replace({"x": 1, "NO": 0})


In [64]:
import pandas as pd
import re

# Función para limpiar texto
def limpiar_texto(texto):
    texto = texto.lower()  # Convertir a minúsculas
    texto = re.sub(r"[^a-zA-ZÑáéíóúñü\s]", "", texto)  # Eliminar caracteres especiales
    texto = re.sub(r"\s+", " ", texto)  # Eliminar múltiples espacios
    texto = texto.strip()  # Eliminar espacios al principio y al final
    texto = re.sub(r"\d+", "", texto)  # Eliminar números
    texto = re.sub(r"\b\w{1,2}\b", "", texto)  # Eliminar palabras de 1 a 2 caracteres

    # reemplazar acentos
    texto = texto.replace("á", "a")
    texto = texto.replace("é", "e")
    texto = texto.replace("í", "i")
    texto = texto.replace("ó", "o")
    texto = texto.replace("ú", "u")
    texto = texto.replace("ñ", "n")
    texto = texto.replace("ü", "u")
    texto = texto.replace("Ñ", "n")


    return texto




In [65]:
# Aplicar la limpieza a la columna 'Titulo'
df["Texto_limpio"] = df["Texto_full"].apply(limpiar_texto)

i=0
for i in df.index[:10]:
  print(df["Texto_limpio"][i])

df.shape


plataforma web app tarjeta pircana  requiere implementar una plataforma tecnologica que permita   municipalidad administrar gestionar  controlar las instancias  interaccion con  comunidad con  objetivo  alcanzar una mejor comunicacion participacion  retroalimentacion  las actividades desarrolladas   territorio  acuerdo  los documentos adjuntos
eort plataforma informatica atencion usuarios  servicio nacional  pesca  acuicultura  adelante sernapesca requiere adquirir una plataforma informatica para  unidad  atencion  usuarios con  fin  permitir  interaccion con usuario  traves  multiples canales  comunicacion como son las llamadas telefonicas chat redes sociales correo electronico etc
servicio produccion  desarrollo  eventos para actividad lanzamiento temporada estival nan
desarrollo del diseno conceptual del indice itps desarrollo del diseno conceptual del indice  transformacion productiva sostenible itps
 servicio  capacitacion curso deteccion  alteraciones del neurodesarrollo   contro

(545, 17)

In [114]:
# SACAR STOP WORDS

nltk.download("stopwords")

stop_words = set(stopwords.words("spanish"))
stop_words.add("asi") #Para agregar nuevas stop word al set
stop_words.add("demas")
stop_words.add("dentro")

df["Texto_limpio_stop"] = df["Texto_limpio"].apply(lambda x: " ".join([word for word in x.split() if word.lower() not in stop_words]))

df.shape

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


(545, 20)

In [116]:
# LEMATIZAR

nlp = spacy.load("es_core_news_sm")  # Cargar el modelo de lenguaje en español

def lematizar_texto(texto):
    doc = nlp(texto)  # Procesar el texto con spaCy
    #print("doc",doc)
    # Explorar los tokens del texto
    #for token in doc:
    #    print(f"Palabra: {token.text}, Lema: {token.lemma_}, Tipo: {token.pos_}")

    return " ".join([token.lemma_ for token in doc])  # Extraer los lemas de cada palabra




In [117]:
#vectorizer.get_feature_names_out()[545:750]

In [118]:
tx = "los niños juegan en los parques parque acuatico acuatica"
txl=lematizar_texto(tx)
print(txl)

el niño jugar en el parque parque acuatico acuatico


In [119]:
df["Texto_Lematizado"] = df["Texto_limpio_stop"].apply(lematizar_texto)  # Lematizar cada fila
df["Texto"] = df["Texto_Lematizado"].apply(limpiar_texto)
df

,CodigoExterno,Nombre,CodigoEstado,FechaCierre,Descripcion,CodigoCategoria,Categoria,CodigoProducto,NombreProducto,CodigoOrganismo,NombreOrganismo,RutUnidad,ComunaUnidad,RegionUnidad,go,Texto_full,Texto_limpio,Texto_limpio_stop,Texto_Lematizado,Texto
230,2719-77-LE24,Plataforma WEB APP Tarjeta Pircana,8,2024-12-03 16:00:00,Se requiere implementar una plataforma tecnoló...,-2.053075,Productos de papel / Productos de papel / Pape...,-2.052599,Tarjetas de socio o libro de biblioteca,100072.0,I MUNICIPALIDAD DE PIRQUE,69.072.200-3,Pirque,Región Metropolitana de Santiago,1.0,Plataforma WEB APP Tarjeta Pircana Se requiere...,plataforma web app tarjeta pircana requiere i...,plataforma web app tarjeta pircana requiere im...,plataforma web app tarjeta pircán requerir imp...,plataforma web app tarjeta pircan requerir imp...
351,701-24-LE24,E1O2RT9 PLATAFORMA INFORMÁTICA ATENCIÓN USUARIOS,8,2024-12-04 15:00:00,"El Servicio Nacional de Pesca y Acuicultura, e...",-0.770708,"Tecnologías de la información, telecomunicacio...",-0.770640,Software de centro de llamadas o soporte técni...,7008.0,SERV NACIONAL DE PESCA,60.701.002-1,Valparaíso,Región de Valparaíso,1.0,E1O2RT9 PLATAFORMA INFORMÁTICA ATENCIÓN USUARI...,eort plataforma informatica atencion usuarios ...,eort plataforma informatica atencion usuarios ...,eort plataforma informatico atencion usuario s...,eort plataforma informatico atencion usuario s...
382,4284-375-E224,Servicio producción y desarrollo de eventos pa...,6,2025-01-03 11:00:00,NaN,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.0,Servicio producción y desarrollo de eventos pa...,servicio produccion desarrollo eventos para ...,servicio produccion desarrollo eventos activid...,servicio produccion desarrollo eventos activid...,servicio produccion desarrollo eventos activid...
402,1098710-58-LE24,DESARROLLO DEL DISEÑO CONCEPTUAL DEL ÍNDICE ITPS,15,2025-01-03T16:28:50.44,DESARROLLO DEL DISEÑO CONCEPTUAL DEL ÍNDICE DE...,0.852966,"Servicios profesionales, administrativos y con...",0.852519,Otras evaluaciones (Gestión y diseño instituci...,1689493.0,"SUBSECRETARIA DE CIENCIA, TECNOLOGIA, CONOCIMI...",62.000.740-4,Santiago,Región Metropolitana de Santiago,0.0,DESARROLLO DEL DISEÑO CONCEPTUAL DEL ÍNDICE IT...,desarrollo del diseno conceptual del indice it...,desarrollo diseno conceptual indice itps desar...,desarrollo diseno conceptual indice itps desar...,desarrollo diseno conceptual indice itps desar...
412,1658-865-LE24,P.334-2024 SERVICIO DE CAPACITACIÓN: CURSO DET...,7,2025-01-03T13:12:56.303,"P.334-2024 ""SERVICIO DE CAPACITACIÓN: CURSO DE...",1.117617,"Educación, formación, entrenamiento y capacita...",1.117087,Capacitación a empleados,7321.0,I MUNICIPALIDAD DE TEMUCO,69.190.700-7,Temuco,Región de la Araucanía,0.0,P.334-2024 SERVICIO DE CAPACITACIÓN: CURSO DET...,servicio capacitacion curso deteccion alter...,servicio capacitacion curso deteccion alteraci...,servicio capacitacion curso deteccion alteraci...,servicio capacitacion curso deteccion alteraci...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46999,3922-9-LE25,Plataformas de Gestión Educativa DAEM Villarrica,6,2025-05-02 15:15:00,NaN,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.0,Plataformas de Gestión Educativa DAEM Villarri...,plataformas gestion educativa daem villarrica...,plataformas gestion educativa daem villarrica nan,plataforma gestion educativo daem villarrico nar,plataforma gestion educativo daem villarrico nar
47038,4888-6-LE25,Produccion integral desarrollo de 6 iniciativa...,6,2025-05-02 15:01:00,NaN,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.0,Produccion integral desarrollo de 6 iniciativa...,produccion integral desarrollo iniciativas nan,produccion integral desarrollo iniciativas nan,produccion integral desarrollo iniciativo nar,produccion integral desarrollo iniciativo nar
47244,885-36-L125,Adquisición de servicios de Soporte y Mantenim...,7,2025-05-02T16:04:02.323,NaN,0.000000,NaN,0.000000,NaN,NaN,NaN,N

In [120]:
# Preprocesamiento de datos
X = df[["Texto", "CodigoProducto", "CodigoCategoria"]]  # Características
y = df["go"]   # Variable objetivo

In [72]:
X

,Texto,CodigoProducto,CodigoCategoria
230,plataforma web app tarjeta pircan requerir imp...,-2.052599,-2.053075
351,eort plataforma informatico atencion usuario s...,-0.770640,-0.770708
382,servicio produccion desarrollo eventos activid...,0.000000,0.000000
402,desarrollo diseno conceptual indice itps desar...,0.852519,0.852966
412,servicio capacitacion curso deteccion alteraci...,1.117087,1.117617
...,...,...,...
46999,plataforma gestion educativo daem villarrico nar,0.000000,0.000000
47038,produccion integral desarrollo iniciativo nar,0.000000,0.000000
47244,adquisicion servicio soporte mantenimiento pla...,0.000000,0.000000
47407,produccion desarrollo software audiovisual nue...,0.000000,0.000000


In [73]:
print(y)

230      1.0
351      1.0
382      0.0
402      0.0
412      0.0
        ... 
46999    0.0
47038    0.0
47244    0.0
47407    0.0
47441    0.0
Name: go, Length: 545, dtype: float64


In [74]:
vectorizer = TfidfVectorizer(max_features=2000)  # Representación TF-IDF
X_tfidf = vectorizer.fit_transform(X["Texto"]).toarray()

print(vectorizer.get_feature_names_out())

X_tfidf_df = pd.DataFrame(X_tfidf, columns=vectorizer.get_feature_names_out())  # Crear DataFrame con palabras clave
X_tfidf_df_alineado = X_tfidf_df.set_index(X.index)

X = pd.concat([X, X_tfidf_df_alineado], axis=1)  # Unir datos originales y TF-IDF

X.shape

['abastecimiento' 'accion' 'aclaracion' ... 'zbc' 'zendesk' 'zona']


(545, 2003)

In [75]:
X

,Texto,CodigoProducto,CodigoCategoria,abastecimiento,accion,aclaracion,acompanamiento,aconcaguo,acondicionado,actividad,...,vulneracion,vvff,wake,web,ybar,yumbel,zaldivar,zbc,zendesk,zona
230,plataforma web app tarjeta pircan requerir imp...,-2.052599,-2.053075,0.0,0.0,0.0,0.0,0.0,0.0,0.130831,...,0.0,0.0,0.0,0.133086,0.0,0.0,0.0,0.0,0.0,0.0
351,eort plataforma informatico atencion usuario s...,-0.770640,-0.770708,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
382,servicio produccion desarrollo eventos activid...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.281047,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
402,desarrollo diseno conceptual indice itps desar...,0.852519,0.852966,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
412,servicio capacitacion curso deteccion alteraci...,1.117087,1.117617,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46999,plataforma gestion educativo daem villarrico nar,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
47038,produccion integral desarrollo iniciativo nar,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
47244,adquisicion servicio soporte mantenimiento pla...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
47407,produccion desarrollo software audiovisual nue...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [76]:
# copias X a X2
X2 = X.copy()

# Eliminar columnas de X2
X2 = X2.drop(["Texto"], axis=1)

X2

,CodigoProducto,CodigoCategoria,abastecimiento,accion,aclaracion,acompanamiento,aconcaguo,acondicionado,actividad,activo,...,vulneracion,vvff,wake,web,ybar,yumbel,zaldivar,zbc,zendesk,zona
230,-2.052599,-2.053075,0.0,0.0,0.0,0.0,0.0,0.0,0.130831,0.0,...,0.0,0.0,0.0,0.133086,0.0,0.0,0.0,0.0,0.0,0.0
351,-0.770640,-0.770708,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
382,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.281047,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
402,0.852519,0.852966,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
412,1.117087,1.117617,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46999,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
47038,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
47244,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
47407,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [77]:
X2.shape


(545, 2002)

In [78]:
X2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 545 entries, 230 to 47441
Columns: 2002 entries, CodigoProducto to zona
dtypes: float64(2002)
memory usage: 8.3 MB


In [79]:
# Verificar si hay valores NaN o Inf en el DataFrame
print(df.isnull().sum())  # Revisa valores NaN
print("......")
print(df.isin([np.inf, -np.inf]).sum())  # Revisa valores Inf

CodigoExterno          0
Nombre                 0
CodigoEstado           0
FechaCierre            0
Descripcion          418
CodigoCategoria        0
Categoria            418
CodigoProducto         0
NombreProducto       418
CodigoOrganismo      418
NombreOrganismo      418
RutUnidad            418
ComunaUnidad         427
RegionUnidad         418
go                     0
Texto_full             0
Texto_limpio           0
Texto_limpio_stop      0
Texto_Lematizado       0
Texto                  0
dtype: int64
......
CodigoExterno        0
Nombre               0
CodigoEstado         0
FechaCierre          0
Descripcion          0
CodigoCategoria      0
Categoria            0
CodigoProducto       0
NombreProducto       0
CodigoOrganismo      0
NombreOrganismo      0
RutUnidad            0
ComunaUnidad         0
RegionUnidad         0
go                   0
Texto_full           0
Texto_limpio         0
Texto_limpio_stop    0
Texto_Lematizado     0
Texto                0
dtype: int64


In [80]:
# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.2, random_state=42)

# Definir la arquitectura del modelo
modelo = keras.Sequential([
    keras.layers.Dense(128, activation="relu", input_shape=(X_train.shape[1],)),
    keras.layers.Dropout(0.3),  # Apaga el 30% de las neuronas
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dropout(0.3),  # Apaga el 30% de las neuronas
    keras.layers.Dense(1, activation="sigmoid")  # Salida binaria
])



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [81]:
modelo.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 128)            │       256,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 264,705 (1.01 MB)

 Trainable params: 264,705 (1.01 MB)

 Non-trainable params: 0 (0.00 B)

In [82]:
# Compilar el modelo
modelo.compile(optimizer=Adam(learning_rate=0.00005), loss="binary_crossentropy", metrics=["accuracy"])


# Entrenar el modelo
modelo.fit(X_train, y_train, epochs=30, batch_size=16, validation_data=(X_test, y_test))

Epoch 1/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.3631 - loss: 0.6995 - val_accuracy: 0.4404 - val_loss: 0.6954
Epoch 2/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5238 - loss: 0.6919 - val_accuracy: 0.7156 - val_loss: 0.6877
Epoch 3/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7082 - loss: 0.6843 - val_accuracy: 0.8257 - val_loss: 0.6801
Epoch 4/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7912 - loss: 0.6771 - val_accuracy: 0.8257 - val_loss: 0.6723
Epoch 5/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8183 - loss: 0.6723 - val_accuracy: 0.8257 - val_loss: 0.6643
Epoch 6/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8101 - loss: 0.6639 - val_accuracy: 0.8257 - val_loss: 0.6561
Epoch 7/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8005 - loss: 0.6568 - val_accuracy: 0.8257 - val_loss: 0.6477
Epoch 8/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8163 - loss: 0.6453 - val_accuracy: 0.8257 - val_loss

In [88]:
predicciones = modelo.predict(X_test)

# Convertir probabilidades en etiquetas (si usaste activación sigmoid)
predicciones_binarias = (predicciones > 0.25).astype(int)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step 


In [84]:
predicciones

array([[0.27605796],
       [0.26349387],
       [0.3220471 ],
       [0.19412275],
       [0.20626396],
       [0.21366295],
       [0.2635902 ],
       [0.20385154],
       [0.2344962 ],
       [0.26726046],
       [0.16893002],
       [0.26875976],
       [0.2507496 ],
       [0.33690357],
       [0.24924238],
       [0.21913634],
       [0.18781199],
       [0.2081862 ],
       [0.27937073],
       [0.21627676],
       [0.16524091],
       [0.22792204],
       [0.2654143 ],
       [0.2182583 ],
       [0.19120575],
       [0.21211039],
       [0.17807427],
       [0.2911608 ],
       [0.24471784],
       [0.28794047],
       [0.2732167 ],
       [0.18486957],
       [0.19449335],
       [0.24278632],
       [0.20646094],
       [0.21580595],
       [0.28730774],
       [0.31896758],
       [0.22190014],
       [0.20277403],
       [0.18992908],
       [0.20311612],
       [0.15823235],
       [0.18016474],
       [0.29507777],
       [0.23493458],
       [0.18851006],
       [0.198

In [90]:
n=len(y_test)
print(n)
for i in y_test.index:  # Iterate through the actual index values
  if y_test[i] == 1:
    pos=X_test.index.get_loc(i)
    print(X.loc[i,"Texto"])
    print("Predic:", predicciones_binarias[y_test.index.get_loc(i)], predicciones[y_test.index.get_loc(i)], "---- Real:", y_test[i])


109
soporte desarrollo mantencion software erp nar
Predic: [1] [0.27605796] ---- Real: 1.0
servicio gestion documental concurso cargo vacante planta municipal gestion personal nar
Predic: [1] [0.3220471] ---- Real: 1.0
plataforma tecnologico integrado atencion vecino nan
Predic: [0] [0.21366295] ---- Real: 1.0
sistema gestion documental nar
Predic: [1] [0.33690357] ---- Real: 1.0
arreir plataforma digital tramit nar
Predic: [0] [0.24924238] ---- Real: 1.0
adquisicion gestor documental penaflor nar
Predic: [1] [0.2654143] ---- Real: 1.0
adquisicion sistema gestion documental meses municipalidad empedrado servicio arreir bodego insumos ornamentacion documento administracion segun termino referencia adjunto
Predic: [0] [0.19449335] ---- Real: 1.0
software web gestion documental nar
Predic: [1] [0.31896758] ---- Real: 1.0
adquisicion equipamiento desarrollo plataforma software educativo nar
Predic: [0] [0.18016474] ---- Real: 1.0
servicio plataforma tecnologico municipalidad recoleta llama

In [55]:
from sklearn.metrics import classification_report

print(classification_report(y_test, predicciones_binarias))


              precision    recall  f1-score   support

         0.0       0.93      0.84      0.88        90
         1.0       0.48      0.68      0.57        19

    accuracy                           0.82       109
   macro avg       0.70      0.76      0.72       109
weighted avg       0.85      0.82      0.83       109

